# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
path1 = "D:/FinUniver/Технологии обработки больших данных/Семинары/06_database/06_database_data/"
path2 = "D:/FinUniver/Технологии обработки больших данных/Семинары/05_data_files_2/data/"
path3 = "D:/FinUniver/Технологии обработки больших данных/Семинары/02_pandas/data/"

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [3]:
conn = sqlite3.connect(path1 + "Chinook_Sqlite.sqlite")
cur = conn.cursor()

cur.execute('''
    SELECT firstname, lastname 
    FROM Customer 
    WHERE country = ?
''', ('Canada', ))

# 1. проитерироваться по cur
# 2. .fetchone(), .fetchall(), .fetchmany(n)

print(cur.fetchmany(3))

cur.close()
conn.close()

[('François', 'Tremblay'), ('Mark', 'Philips'), ('Jennifer', 'Peterson')]


2. Найти и вывести на экран названия всех альбомов группы Accept

In [15]:
conn = sqlite3.connect(path1 + "Chinook_Sqlite.sqlite")
cur = conn.cursor()

cur.execute('''
    SELECT artistid 
    FROM Artist 
    WHERE name = ?
''', ('Accept', ))
ids = cur.fetchone()

cur.execute('''
    SELECT title 
    FROM Album 
    WHERE artistid = ?
''', (ids[0], ))

print(cur.fetchall())

cur.close()
conn.close()

[('Balls to the Wall',), ('Restless and Wild',)]


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [13]:
conn = sqlite3.connect(path1 + "PI19-4.db")
cur = conn.cursor()

cur.execute("""
    CREATE TABLE Student(id INTEGER, name VARCHAR(50))
""")
cur.execute("""
    INSERT INTO Student(id, name) VALUES (1, 'Dmitriy')
""")
conn.commit()

cur.close()
conn.close()

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [4]:
conn = sqlite3.connect(path1 + "recipes.db")
cur = conn.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [5]:
# cur.execute("DROP TABLE Recipe")
# conn.commit()
cur.execute("""
    CREATE TABLE Recipe(
        id INTEGER PRIMARY KEY, 
        name VARCHAR(50) NOT NULL, 
        minutes INTEGER,
        submitted DATE NOT NULL,
        desciption VARCHAR(200),
        n_ingredients INTEGER
    )
""")

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [6]:
# cur.execute("DROP TABLE Review")
# conn.commit()
cur.execute("""
    CREATE TABLE Review(
        id INTEGER PRIMARY KEY, 
        user_id INTEGER NOT NULL,
        recipe_id INTEGER NOT NULL,
        date DATE NOT NULL,
        rating INTEGER NOT NULL,
        review VARCHAR(250),
        FOREIGN KEY (recipe_id) REFERENCES Recipe(id)
    )
""")

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [7]:
reviews_sample = pd.read_csv(path3 + 'reviews_sample.csv', sep=',')
reviews_sample = reviews_sample.set_index('Unnamed: 0')
reviews_sample

,user_id,recipe_id,date,rating,review
Unnamed: 0,,,,,
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
910362,35106,31322,2003-01-03,4,I forgot to add skim milk but it still tasted ...
212649,404333,199579,2006-12-10,5,"Made this for dinner it was so excellent, fina..."
815389,162888,16067,2005-12-09,5,"When I snapped the picture, I forgot to review..."
642377,89831,33715,2007-07-03,5,This was good combination of flavors but I wil...


In [8]:
recipes_sample_with_tags_ingredients = pd.read_csv(path2 + 'recipes_sample_with_tags_ingredients.csv', sep=',')
recipes_sample_with_tags_ingredients

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_tags,tags,ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,4,an original recipe created by chef scott meska...,18,25,weeknight;time-to-make;course;main-ingredient;...,unsalted butter*carrot*onion*celery*broccoli s...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3,my children and their friends ask for my homem...,3,31,15-minutes-or-less;time-to-make;course;prepara...,milk*frozen juice concentrate*plain yogurt
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5,"these were so go, it surprised even me.",8,17,30-minutes-or-less;time-to-make;course;main-in...,onion*frozen chopped spinach*eggs*garlic powde...
3,italian gut busters,35173,45,22724,2002-07-27,7,my sister-in-law made these for us at a family...,9,11,60-minutes-or-less;time-to-make;course;prepara...,sandwich bun*good seasonings italian salad dre...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4,i think a fondue is a very romantic casual din...,12,19,30-minutes-or-less;time-to-make;course;main-in...,beef steaks*vegetable oil*spicy mustard*fresh ...
5,mennonite corn fritters,44045,15,41706,2002-10-25,6,ok - my heritage has been revealed. :) these a...,8,29,15-minutes-or-less;time-to-make;course;main-in...,eggs*fresh corn*flour*salt*pepper*baking powde...
6,open sesame noodles,107229,28,173674,2004-12-30,8,this is a very versatile and widely enjoyed pa...,12,13,30-minutes-or-less;time-to-make;course;main-in...,vegetable oil*vermicelli*rice vinegar*reduced ...
7,say what banana sandwich,95926,5,118163,2004-07-20,4,you just have to try it to believe it.,3,30,15-minutes-or-less;time-to-make;course;main-in...,white bread*mayonnaise*bananas
8,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,12,this is the recipe that we use at my school ca...,11,9,60-minutes-or-less;time-to-make;cuisine;prepar...,eggs*margarine*brown sugar*salt*white sugar*va...
9,412 broccoli casserole,306168,40,50969,2008-05-30,6,since there are already 411 recipes for brocco...,9,10,60-minutes-or-less;time-to-make;course;main-in...,milk*garlic powder*salt*frozen broccoli cuts*c...


In [9]:
for index, row in reviews_sample.iterrows():
    cur.execute("""
        INSERT INTO Review(id, user_id, recipe_id, date, rating, review) 
        VALUES (?, ?, ?, ?, ?, ?)
    """, (index, row[0], row[1], row[2], row[3], row[4]))
conn.commit()

In [10]:
for index, row in recipes_sample_with_tags_ingredients.iterrows():
    cur.execute("""
        INSERT INTO Recipe(id, name, minutes, submitted, desciption, n_ingredients) 
        VALUES (?, ?, ?, ?, ?, ?)
    """, (row[1], row[0], row[2], row[4], row[6], row[7]))
conn.commit()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [11]:
cur.execute('''
    SELECT * 
    FROM Recipe 
    WHERE n_ingredients = ?
''', (10, ))
print(cur.fetchmany(5))

[(176, 'cola cake', 55, '1999-08-22', None, 10), (246, 'lee s hot crab dip', 45, '1999-09-01', "lee's hot crab dip", 10), (289, 'feijoada  brazilian bean soup  ii', 150, '1999-10-04', 'brazilian bean soup', 10), (373, 'tiramisu   balducci s in new york city', 95, '1999-08-17', 'this is an adopted recipe. i shall be trying it out and then post any additional information if needs be. if you are going to give any or no stars, please at least give the reason why so that you can help me improve the recipe. thank you.', 10), (544, 'greek mushroom salad', 38, '1999-08-10', None, 10)]


6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [12]:
cur.execute('''
    SELECT name, MAX(minutes) 
    FROM Recipe 
''')
print(cur.fetchone())

('strawberry liqueur', 129615)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [37]:
try:
    id_recipe = int(input("Введите индекс рецепта: "))
    
    cur.execute('''
        SELECT *
        FROM Recipe
        WHERE id = ?
    ''', (id_recipe, ))
    result = cur.fetchone()
    
    print(result) if result is not None else print("Рецепт не существует")
except ValueError:
    print("Неверный формат индекса рецепта")

Введите индекс рецепта: 176
(176, 'cola cake', 55, '1999-08-22', None, 10)


8. Найдите кол-во отзывов с рейтингом 5.

In [36]:
cur.execute('''
    SELECT COUNT(*)
    FROM Review
    WHERE rating = ?
''', (5, ))
print(cur.fetchone()[0])

91361


9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [38]:
res = cur.execute('''
    SELECT COUNT(*) OVER()
    FROM recipe rec
    LEFT JOIN review rev
    ON rec.id = rev.recipe_id
    GROUP BY rec.id
    HAVING 0 = COUNT(rev.rating < ?)
    LIMIT 1
''', (4, ))

print(res.fetchone()[0])

1900


10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [15]:
result = cur.execute("""
    SELECT COUNT(*)
    FROM Recipe
    WHERE minutes >= ?
    AND submitted BETWEEN ? AND ?
""", (15, '2010-01-01', '2010-12-31'))
print(result.fetchone()[0])

1319


11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [41]:
res = cur.execute('''
    SELECT rec.id, rec.name, rev.user_id, rev.date, rev.rating
    FROM recipe rec
    LEFT JOIN review rev
    ON rec.id = rev.recipe_id
    WHERE rec.n_ingredients >= ?
    ORDER BY rec.id
''', (3, ))

print(res.fetchmany(1000))

[(48, 'boston cream pie', 32421, '2002-03-15', 0), (48, 'boston cream pie', 68674, '2004-05-03', 2), (55, 'betty crocker s southwestern guacamole dip', 53959, '2006-01-12', 4), (55, 'betty crocker s southwestern guacamole dip', 165567, '2006-03-31', 5), (55, 'betty crocker s southwestern guacamole dip', 851190, '2010-05-23', 5), (55, 'betty crocker s southwestern guacamole dip', 1060485, '2009-04-07', 5), (66, 'black coffee barbecue sauce', 8679, '2003-03-17', 5), (66, 'black coffee barbecue sauce', 42938, '2002-10-21', 4), (66, 'black coffee barbecue sauce', 124416, '2010-07-08', 5), (66, 'black coffee barbecue sauce', 133174, '2010-07-04', 5), (66, 'black coffee barbecue sauce', 136004, '2008-06-05', 5), (66, 'black coffee barbecue sauce', 136813, '2007-04-14', 5), (66, 'black coffee barbecue sauce', 176798, '2010-10-02', 5), (66, 'black coffee barbecue sauce', 314792, '2008-02-04', 5), (66, 'black coffee barbecue sauce', 330545, '2009-01-02', 5), (66, 'black coffee barbecue sauce', 

In [ ]:
cur.close()
conn.close()